In [292]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
!pip install pymatgen
!pip install mp_api
import requests
import json

In [293]:
update_IDs = False
target_col = 'Log_rate'
cross_validation = True

df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset export.xlsx",sheet_name='Photocatalytic dataset')

In [294]:
df

,Perovskite,Hill formula,Interlayer space composition,"Bandgap, eV",DOI,Materials Project ID,COD_ID,Springer_ID,Z,Z_MP,...,"CatW, g/L","Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)"
0,K4Nb6O17,K4 Nb6 O17,NaN,3.50,10.1007/BF00765319,mp-560692,1001842,NaN,4.0,4.0,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,130.000000
1,KLaNb2O7,K La Nb2 O7,NaN,3.20,10.1007/BF00765319,mp-1223501,1545643,NaN,8.0,1.0,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,28.000000
2,RbLaNb2O7,La Nb2 O7 Rb,NaN,3.35,10.1007/BF00765319,mp-553965,-1,NaN,1.0,2.0,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,60.000000
3,CsLaNb2O7,Cs La Nb2 O7,NaN,3.30,10.1007/BF00765319,mp-553248,2004917,NaN,1.0,1.0,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,12.000000
4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,3.35,10.1007/BF00765319,mp-557195,1521061,NaN,8.0,4.0,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,TiO2,NaN,NaN,NaN,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,38.181818
1085,TiO2,NaN,NaN,NaN,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,36.363636
1086,TiO2,NaN,NaN,NaN,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909
1087,TiO2,NaN,NaN,NaN,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909


In [295]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 78 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            1089 non-null   object 
 1   Hill formula                          730 non-null    object 
 2   Interlayer space composition          5 non-null      object 
 3   Bandgap, eV                           924 non-null    float64
 4   DOI                                   1084 non-null   object 
 5   Materials Project ID                  901 non-null    object 
 6   COD_ID                                876 non-null    object 
 7   Springer_ID                           181 non-null    object 
 8   Z                                     694 non-null    float64
 9   Z_MP                                  832 non-null    float64
 10  Z_COD                                 836 non-null    float64
 11  Z_Springer       

In [296]:
df.describe()

,"Bandgap, eV",Z,Z_MP,Z_COD,Z_Springer,"d,A",a_MP,b_MP,c_MP,a_COD,...,Oxygen_concentration avg,Packing fraction avg,CalcT(K),Calc time (h),"CatW, g/L","Alcohol, %","Power, W","Wave length min, nm","Temperature, K","Rate, umol/(g*h)"
count,924.000000,694.000000,832.000000,836.000000,836.000000,807.000000,827.000000,354.000000,354.000000,832.000000,...,832.000000,832.000000,1010.000000,973.000000,987.000000,966.000000,994.000000,689.000000,540.000000,989.000000
mean,3.296839,2.275216,1.260817,1.281100,0.151715,13.707422,3.084536,7.406617,14.790353,3.040421,...,inf,0.238767,1287.020792,19.338386,1.428618,5.628230,306.726358,288.388970,294.435185,6914.999659
std,0.639285,1.762830,3.460696,2.116075,0.642689,8.610255,4.236046,3.399894,8.264087,6.154639,...,NaN,0.257556,222.542464,25.264723,1.324191,8.781436,198.606250,88.814702,13.175195,21648.175384
min,0.010000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,3.840000,3.904344,0.000000,...,0.000000,0.000000,453.000000,0.000000,0.000000,0.000000,100.000000,200.000000,288.000000,0.000000
25%,3.080000,1.000000,0.000000,0.000000,0.000000,10.548300,0.000000,3.950634,10.502783,0.000000,...,0.037279,0.000000,1223.000000,10.000000,0.500000,0.000000,125.000000,220.000000,288.000000,15.500000
50%,3.410000,2.000000,0.000000,0.000000,0.000000,14.733000,0.000000,7.415443,12.879040,0.000000,...,0.042938,0.000000,1373.000000,12.000000,1.000000,1.000000,300.000000,220.000000,288.000000,166.000000
75%,3.600000,4.000000,1.000000,2.000000,0.000000,17.900000,5.801718,7.938952,15.513552,3.899000,...,0.046684,0.533675,1423.000000,24.000000,2.000000,10.000000,450.000000,400.000000,293.000000,1680.000000
max,5.500000,16.000000,50.000000,16.000000,4.000000,38.670000,22.855297,20.551678,48.809404,41.857000,...,inf,0.657822,1873.000000,200.000000,10.000000,90.000000,1000.000000,660.000000,333.000000,156000.000000


In [297]:
df.columns

Index(['Perovskite', 'Hill formula', 'Interlayer space composition',
       'Bandgap, eV', 'DOI', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'Z', 'Z_MP', 'Z_COD', 'Z_Springer', 'a, A', 'b, A', 'c, A',
       'Symmetry group', 'd,A', 'a_MP', 'b_MP', 'c_MP', 'a_COD', 'b_COD',
       'c_COD', 'a_Springer', 'b_Springer', 'c_Springer',
       'Number of octahedrons on a layer', 'Valence electrons', 'Volume',
       'Volume_MP', 'Volume_COD', 'Volume_Springer',
       'Valence Electrons Density', 'Valence Electrons Density_MP',
       'Valence Electrons Density_COD', 'Springer_Valence Electrons Density',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'MagpieData minimum Electronegativity',
       'MagpieData maximum Electronegativity',
       'MagpieData range Electronegativ

In [298]:
init_columns = ['Perovskite', 'Interlayer space composition',
       'Bandgap, eV', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'Z', 'a, A', 'b, A', 'c, A',
       'Symmetry group', 'd,A',
       'Number of octahedrons on a layer', 'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'Nitrogen', 'Promotion method', 'Promoter', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Light type', 'Power, W', 'Wave length min, nm',
       'Cell material', 'Temperature, K', 'Rate, umol/(g*h)']

In [299]:
df = df[init_columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1089 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       924 non-null    float64
 3   Materials Project ID              901 non-null    object 
 4   COD_ID                            876 non-null    object 
 5   Springer_ID                       181 non-null    object 
 6   Z                                 694 non-null    float64
 7   a, A                              735 non-null    object 
 8   b, A                              717 non-null    object 
 9   c, A                              717 non-null    object 
 10  Symmetry group                    494 non-null    object 
 11  d,A                               807 non-null    float64
 12  Number

#Formatting


In [300]:
subs_map = {
    "Ph": "C6H5",
    "Bn": "C7H7",
    "Pr": "C3H7",
    "Bu": "C4H9",
    "Hx": "C6H13",
    "Me": "CH3",
    "Et": "C2H5",
    "Oc": "C8H17",
    "Dc": "C10H21",
}

import re

def expand_substituents(formula):
    if pd.isna(formula):
        return formula

    for abbr, full in subs_map.items():
        formula = re.sub(rf'{abbr}', full, formula)
    return formula

In [301]:
print(df.shape[0])
df = df[~df['Perovskite'].str.contains("Nx", na=False)]
df = df[~df['Perovskite'].str.contains("Ox", na=False)]
print(df.shape[0])

1089
1068


In [302]:
df['Perovskite'] = df['Perovskite'].apply(expand_substituents)
#print(expand_substituents('HNdTiO4*MeNH2'))

In [303]:
df = df.replace("-", np.nan)
df["a, A"] = df["a, A"].astype("float64")
df["b, A"] = df["b, A"].astype("float64")
df["c, A"] = df["c, A"].astype("float64")
df["Number of octahedrons on a layer"] = df["Number of octahedrons on a layer"].astype("float64")

/tmp/ipython-input-3678714408.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [304]:
df["Nitrogen"] = df["Nitrogen"].fillna(False)
df["Nitrogen"]
df["Nitrogen"].astype("bool")
df["Nitrogen"].value_counts()

/tmp/ipython-input-1068474039.py:1: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,count
Nitrogen,
False,1021
True,47


In [305]:
df["Promoter, w%"] = df["Promoter, w%"].fillna(0)
df["Promoter, w%"].value_counts()

,count
"Promoter, w%",
0.000,489
1.000,291
0.100,81
0.500,72
0.150,27
0.200,24
0.300,16
3.000,15
2.000,10


In [306]:
df["Surface area, m2/g"] = df["Surface area, m2/g"].astype("float64")
df["Surface area, m2/g"].value_counts()

#df['Materials Project ID'] = df['Materials Project ID'].astype(str)
print(df['Materials Project ID'].dtype)

object


In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1068 entries, 0 to 1088
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1068 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       907 non-null    float64
 3   Materials Project ID              880 non-null    object 
 4   COD_ID                            855 non-null    object 
 5   Springer_ID                       181 non-null    object 
 6   Z                                 686 non-null    float64
 7   a, A                              723 non-null    float64
 8   b, A                              705 non-null    float64
 9   c, A                              705 non-null    float64
 10  Symmetry group                    488 non-null    object 
 11  d,A                               788 non-null    float64
 12  Number of o

In [308]:
df.to_excel("checkpoint_formatting.xlsx")


#Data cleaning

##Missing data handling

In [309]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,1063
"Bandgap, eV",161
Materials Project ID,188
COD_ID,213
Springer_ID,887
Z,382
"a, A",345
"b, A",363
"c, A",363


In [310]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)'],
      dtype='object')

In [311]:
df.dropna(subset=['Perovskite', 'Bandgap, eV',
       'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'CatW, g/L', 'Light type',
       'Power, W',
       'Rate, umol/(g*h)'], inplace=True)
df.shape[0]

745

In [312]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,740
"Bandgap, eV",0
Materials Project ID,104
COD_ID,106
Springer_ID,648
Z,243
"a, A",227
"b, A",233
"c, A",233


In [313]:
def calculate_average_specific_area(dataframe):
  total=0;
  n=0;
  for index, row in df.iterrows():
    area =  row['Surface area, m2/g']
    if(np.isnan(area) or area > 55):
      continue
    total += area
    n+=1
  return total/n

In [314]:
avr_specific_area = calculate_average_specific_area(df)
print(avr_specific_area)
df["Surface area, m2/g"] = df["Surface area, m2/g"].fillna(avr_specific_area)

df["Alcohol, %"] = df["Alcohol, %"].fillna(0)
df["Wave length min, nm"] = df["Wave length min, nm"].fillna(df["Wave length min, nm"].min())
df["Cell material"] = df["Cell material"].fillna(df["Cell material"].mode()[0])
print(df["Cell material"].mode()[0])
df["Temperature, K"] = df["Temperature, K"].fillna(298.15)
df["Materials Project ID"] = df["Materials Project ID"].fillna(-1)
df["COD_ID"] = df["COD_ID"].fillna(-1)
df["Springer_ID"] = df["Springer_ID"].fillna(-1)

7.735057306590246
quartz


In [315]:
df["Promoter"] = df["Promoter"].fillna("No promoter")
df["Promotion method"] = df["Promotion method"].fillna("No promotion")

In [316]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,740
"Bandgap, eV",0
Materials Project ID,0
COD_ID,0
Springer_ID,0
Z,243
"a, A",227
"b, A",233
"c, A",233


## Duplicates handling

In [317]:
duplicates = df[ df.duplicated()]
duplicates

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,"CatW, g/L","Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)"


##Handling outliers

In [318]:
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %',
       'Power, W', 'Wave length min, nm', 'Temperature, K']
#columns_outliers_detedection=[]

In [319]:
import plotly.express as px
def plot_destribution(df,column):
  #fig = px.histogram(df, x=column,nbins=40,width=800, height=600, title=column)
  fig = px.violin(df, x=column,width=800, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()
def plot_histogram(df,column):
  fig = px.histogram(df, x=column,nbins=10,width=800, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()

In [320]:
for column in columns_outliers_detedection:
  plot_histogram(df,column)

In [321]:
def IQR_column_outliers_removal(_df, _column):
  num_rows_before = _df.shape[0]
  Q1 = _df[_column].quantile(0.25)
  Q3 = _df[_column].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  df_clean = _df[(_df[_column]>= lower_bound) & (_df[_column] <= upper_bound)]
  num_rows_after = df_clean.shape[0]
  removed = num_rows_before - num_rows_after
  print(f'Column: {_column} removed ouliers: {removed }')
  print(f'min: {lower_bound} max: {upper_bound}')
  return df_clean

In [322]:
for column in columns_outliers_detedection:
  df = IQR_column_outliers_removal(df,column)
print(f'Rows left: {df.shape[0]}')

Column: Bandgap, eV removed ouliers: 101
min: 2.3200000000000003 max: 4.4
Column: Calc time (h) removed ouliers: 95
min: -11.0 max: 45.0
Column: Promoter, w% removed ouliers: 15
min: -1.5 max: 2.5
Column: Surface area, m2/g removed ouliers: 64
min: -0.3088359598853687 max: 12.561393266475616
Column: CatW, g/L removed ouliers: 21
min: -1.75 max: 4.25
Column: Alcohol, % removed ouliers: 61
min: -7.14 max: 11.899999999999999
Column: Power, W removed ouliers: 0
min: -287.5 max: 812.5
Column: Wave length min, nm removed ouliers: 70
min: 170.0 max: 250.0
Column: Temperature, K removed ouliers: 4
min: 272.77500000000003 max: 313.37499999999994
Rows left: 314


#Descriptors calculation



## Material Project IDs


In [323]:
from mp_api.client import MPRester

API_KEY = "GFsoU5OV3dEngGT860TOtWcn35bE4y6l"
with MPRester(API_KEY) as mpr:
    materials = mpr.materials.search(formula="KCa2Nb3O10")
for material in materials:
    print(material.material_id, material.formula_pretty)

Retrieving MaterialsDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

mp-557195 KCa2Nb3O10
mp-1223594 KCa2Nb3O10


In [324]:
def get_material_id_by_formula(f):
  with MPRester(API_KEY) as mpr:
    print(f"Composition: {f}")
    try:
      materials = mpr.materials.search(formula=f)
      if len(materials)>0:
        print(f"Composition: {f} ID is {materials[0].material_id}")
        return str(materials[0].material_id)
      else:
        print(f"Composition: {f} ID is not found")
        return str(-1)
    except Exception as e:
      print(f"Composition: {f} ID is not found")
      return str(-1)

In [325]:
def update_Materials_Project_IDs(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['Materials Project ID']) or row['Materials Project ID']==-1 or row['Materials Project ID']==-2 :
            new_id = get_material_id_by_formula(row['Perovskite'])
            print(f"New ID fetched: {new_id}  for formula {row['Perovskite']}")
            _df.at[index, 'Materials Project ID'] = str(new_id)
  return _df

In [326]:
if update_IDs:
  df = update_Materials_Project_IDs(df)

##HIll's formula

In [327]:
import re
from collections import Counter
def parse_formula(formula):
    """Extracts element counts from a chemical formula."""
    pattern = r"([A-Z][a-z]*)(\d*)"
    matches = re.findall(pattern, formula)

    element_counts = Counter()
    for element, count in matches:
        element_counts[element] += int(count) if count else 1  # Default to 1 if count is missing

    return element_counts
def to_hill_notation(formula):
    """Converts a chemical formula to Hill notation."""
    element_counts = parse_formula(formula)

    # Hill notation rules
    if "C" in element_counts:
        elements_sorted = ["C", "H"] if "H" in element_counts else ["C"]
        remaining_elements = sorted(e for e in element_counts if e not in ["C", "H"])
        elements_sorted.extend(remaining_elements)
    else:
        elements_sorted = sorted(element_counts.keys())
    hill_formula = " ".join(f"{el}{element_counts[el] if element_counts[el] > 1 else ''}" for el in elements_sorted)
    #print(f':{hill_formula}:')
    return hill_formula


In [328]:
formulas = ["H2O", "C6H12O6", "NH3", "Fe2O3", "CH4", "CCl4", "H4C", "KCa2Nb3O10"]
for f in formulas:
    print(f"{f} -> {to_hill_notation(f)}")

H2O -> H2 O
C6H12O6 -> C6 H12 O6
NH3 -> H3 N
Fe2O3 -> Fe2 O3
CH4 -> C H4
CCl4 -> C Cl4
H4C -> C H4
KCa2Nb3O10 -> Ca2 K Nb3 O10


In [329]:
df['Hill formula']=df['Perovskite'].apply(to_hill_notation)
df

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,"Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)",Hill formula
69,RbLaTa2O7,NaN,3.90,mp-541600,-1,-1,1.0,3.885,3.885,11.12,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,6.000000,La O7 Rb Ta2
70,RbC3H7Ta2O7,NaN,3.50,-1,-1,sd_1050391,1.0,3.884,3.884,11.12,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,4.500000,C3 H7 O7 Rb Ta2
71,RbNdTa2O7,NaN,3.80,-1,-1,sd_1050391,1.0,3.846,3.846,11.10,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,235.000000,Nd O7 Rb Ta2
72,RbSmTa2O7,NaN,3.80,-1,-1,sd_1050391,1.0,3.842,3.842,11.11,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,53.000000,O7 Rb Sm Ta2
83,La4Ti3O12,NaN,3.95,mp-3249,1526720,-1,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,Hg lamp,300.0,200.0,quartz,298.15,18.000000,La4 O12 Ti3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,HNdNb2O7,NaN,3.43,-1,-1,sd_1249278,1.0,3.862,3.862,10.04,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,5.102041,H Nb2 Nd O7
1067,LiNdNb2O7,NaN,3.58,-1,-1,sd_1249278,1.0,3.862,3.862,10.22,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,162.627551,Li Nb2 Nd O7
1068,NaNdNb2O7*1.27H2O,NaN,3.57,-1,-1,sd_1249278,1.0,3.862,3.862,10.42,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,29.336735,H2 Na Nb2 Nd O8
1069,RbNdNb2O7,NaN,3.55,-1,-1,sd_1249278,1.0,3.862,3.862,10.93,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,245.535714,Nb2 Nd O7 Rb


##COD_IDs


In [330]:
def get_COD_ID_for_formula(formula):
  print(f"Formula :{formula}:")
  url=f"https://www.crystallography.net/cod/result?formula={formula}&format=json"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    #print("Len: ",len(data))
    if(len(data)>0):
      #print(data[0]['file'])
      return data[0]['file']
  print(-1)
  return -1

def update_COD_ID(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['COD_ID']) or row['COD_ID']==-1 or row['COD_ID']==-2 :
            print(f"New ID fetching start for formula {row['Hill formula']}")
            new_id = get_COD_ID_for_formula(row['Hill formula'])
            print(f"New ID fetched: {new_id}  for formula {row['Hill formula']}")
            _df.at[index, 'COD_ID'] = new_id
  return _df


In [331]:
df.to_excel('checkpoint_Hill_formula.xlsx')
#print(get_COD_ID("O2 Ti"))
if update_IDs:
  df = update_COD_ID(df)

##CIF files extraction

In [332]:
def extract_CIF_from_MP(material_id):
  if pd.isna(material_id) or material_id == "-1" or material_id == -1 or material_id == -2:
     print("Skipping invalid ID:", material_id)
     return
  print("ID: ", material_id)

  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif'
  if os.path.exists(path):
    print(f'CIF file for {material_id} already exist')
    return

  with MPRester(API_KEY) as mpr:
    #data = mpr.materials.search(material_ids=material_id)
    structure = mpr.materials.get_structure_by_material_id(material_id )
    cif_string = structure.to("struct.cif")

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  with open(f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif', 'w') as f:
      f.write(cif_string)
  print("ID: ", material_id, " done!")

In [333]:
import os
def extract_cif_from_COD(COD_ID):
  if(COD_ID==-1 or COD_ID==-2 ):
    return
  print(COD_ID)
  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{COD_ID}.cif'
  if os.path.exists(path):
    print(f'CIF file for {COD_ID} already exist')
    return

  url = f"https://www.crystallography.net/cod/{COD_ID}.cif"
  response = requests.get(url)

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  if response.status_code == 200:
    print("Sucess")
    with open(path, 'w') as f:
        f.write(response.text)
  else:
    print('No CIF')

In [334]:
for material_id in df['COD_ID'].unique():
  extract_cif_from_COD(material_id)

1526720
CIF file for 1526720 already exist
1526777
CIF file for 1526777 already exist
8101224
CIF file for 8101224 already exist
7039788
CIF file for 7039788 already exist
1522043
CIF file for 1522043 already exist
7221084
CIF file for 7221084 already exist
2229871
CIF file for 2229871 already exist
7221013
CIF file for 7221013 already exist
2102087
CIF file for 2102087 already exist
1540895
CIF file for 1540895 already exist
1530317
CIF file for 1530317 already exist
1544432
CIF file for 1544432 already exist
1011128
CIF file for 1011128 already exist
2002850
CIF file for 2002850 already exist
1001022
CIF file for 1001022 already exist
1001842
CIF file for 1001842 already exist
2002197
CIF file for 2002197 already exist
sd_1045310
CIF file for sd_1045310 already exist
sd_0548135
CIF file for sd_0548135 already exist
sd_0548134
CIF file for sd_0548134 already exist
1001177
CIF file for 1001177 already exist
1544433
CIF file for 1544433 already exist
2007573
CIF file for 2007573 already

In [335]:
for material_id in df['Materials Project ID'].unique():
  extract_CIF_from_MP(material_id)

ID:  mp-541600
CIF file for mp-541600 already exist
Skipping invalid ID: -1
ID:  mp-3249
CIF file for mp-3249 already exist
ID:  mp-1228245
CIF file for mp-1228245 already exist
ID:  mp-1228150
CIF file for mp-1228150 already exist
ID:  mp-561133
CIF file for mp-561133 already exist
ID:  mp-3563
CIF file for mp-3563 already exist
ID:  mp-10347
CIF file for mp-10347 already exist
ID:  mp-1221096
CIF file for mp-1221096 already exist
ID:  mp-1104930
CIF file for mp-1104930 already exist
ID:  mp-6259
CIF file for mp-6259 already exist
ID:  mp-7148
CIF file for mp-7148 already exist
ID:  mp-3614
CIF file for mp-3614 already exist
ID:  mp-5308
CIF file for mp-5308 already exist
ID:  mp-990430
CIF file for mp-990430 already exist
ID:  mp-3349
CIF file for mp-3349 already exist
ID:  mp-1204933
CIF file for mp-1204933 already exist
Skipping invalid ID: -2
ID:  mp-1220741
CIF file for mp-1220741 already exist
ID:  mp-4155
CIF file for mp-4155 already exist
ID:  mp-15590
CIF file for mp-15590 al

##Valence electron discriptors

In [336]:
!pip install matminer
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ValenceOrbital
from pymatgen.core.composition import Composition


In [337]:
df['composition_obj'] = df['Hill formula'].apply(Composition)
print("Null compositions",df['composition_obj'].isna().sum())
vo_feat = ValenceOrbital()
df = vo_feat.featurize_dataframe(df, col_id='composition_obj')
df = df.drop('composition_obj', axis=1)

Null compositions 0


/usr/local/lib/python3.12/dist-packages/matminer/featurizers/composition/orbital.py:115: UserWarning:

ValenceOrbital(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.

/usr/local/lib/python3.12/dist-packages/matminer/utils/data.py:326: UserWarning:

MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after f

ValenceOrbital:   0%|          | 0/314 [00:00<?, ?it/s]

##Mulliken electronegativity

In [338]:
from pymatgen.core.periodic_table import Element

def get_mulliken_en(element_symbol):
    el = Element(element_symbol)
    IE = el.ionization_energies[0]  # First ionization energy in eV
    EA = el.electron_affinity       # Electron affinity in eV

    if IE is None or EA is None:
        return None

    return (IE + EA) / 2


def calc_average_electronegativity(formula):
  # Example: For Fe2O3
  comp = Composition(formula)
  # Weighted mean electronegativity
  total_atoms = comp.num_atoms
  mean_en = sum(
    #comp[el] / total_atoms * Element(el).X
    comp[el] / total_atoms * get_mulliken_en(el)
    for el in comp.elements)
  return mean_en

In [339]:
df["Average Mulliken electronegativity"] = df["Hill formula"].apply(calc_average_electronegativity)

##Valence electron density

In [340]:
def split_formula(formula):
  output={}
  elements_with_indexes = formula.split()
  for el in elements_with_indexes:
    #match = re.match(r"([A-Za-z]+)(\d+)$", el)
    match = re.match(r"([A-Za-z]+)(\d+(?:\.\d+)?)$", el)
    if match:
      output[match.group(1)]=float(match.group(2))
    else:
      output[ el]=float(1)
  return output

def get_valence_electrons_number(hill_fomula):
  split = split_formula(hill_fomula)
  print(split)
  v = split.get("O")
  if v == None:
    return 0
  else:
    return 2*v

In [341]:
df["Valence electrons"] = df["Hill formula"].apply(get_valence_electrons_number)

{'La': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'C': 3.0, 'H': 7.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'Nd': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Rb': 1.0, 'Sm': 1.0, 'Ta': 2.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ba': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'Ba': 3.0, 'La': 1.0, 'Nb': 3.0, 'O': 12.0}
{'Nb': 4.0, 'O': 15.0, 'Sr': 5.0}
{'Ba': 5.0, 'Nb': 4.0, 'O': 15.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'Nb': 6.0, 'O': 17.0, 'Rb': 4.0}
{'Ca': 2.0, 'Cs': 1.0, 'O': 10.0, 'Ta': 3.0}
{'Ca': 1.0, 'Cs': 1.0, 'Na': 1.0, 'O': 10.0, 'Ta': 3.0}
{'Ca': 2.0, 'Cs': 1.0, 'H': 1.0, 'Na': 1.0, 'O': 10.0, 'Ta': 3.0}
{'C': 6.0, 'H': 16.0, 'Ca': 2.0, 'N': 1.0, 'O': 10.0, 'Ta': 3.0}
{'Ca': 2.0, 'Cs': 1.0, 'O': 10.0, 'Ta': 3.0}
{'Ca': 1.0, 'Cs': 1.0, 'Na': 1.0, 'O': 10.0, 'Ta': 3.

##CIF files descriptors

In [342]:
from pymatgen.core.structure import Structure

def getCellVolume(ID):
  print("getCellVolume() ID:",ID)
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0

  if(structure == None):
    return 0
  return structure.volume

In [343]:
def getCellZValue(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0
  if(structure == None):
    return 0
  return structure.composition.num_atoms / structure.composition.reduced_composition.num_atoms

In [344]:
def abcExtractionFromMP(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return [0,0,0]
  else:
    return  [0,0,0]

  if(structure == None):
    return  [0,0,0]
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]

In [345]:
df["Volume_MP"]=df["Materials Project ID"].apply(getCellVolume)
df["Z_MP"]=df["Materials Project ID"].apply(getCellZValue)
df['Valence Electrons Density_MP'] = (df['Valence electrons'] * df['Z_MP']) / df['Volume_MP']
df[['a_MP', 'b_MP', 'c_MP']] = df['Materials Project ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: mp-541600
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245
getCellVolume() ID: mp-1228150
getCellVolume() ID: mp-561133
getCellVolume() ID: mp-3563
getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: -1
getCellVolume() ID: mp-10347
getCellVolume() ID: mp-1221096
getCellVolume() ID: mp-1221096
getCellVolume() ID: -1
getCellVolume() ID: mp-10347
getCellVolume() ID: mp-1221096
getCellVolume() ID: mp-1221096
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: -1
getCellVolume() ID: mp-6259
getCellVolume() ID: mp-7148
getCellVolume() ID: mp-7148
getCellVolume() ID: mp-3614
getCellVolume() ID: -1
getCellVolume() ID: mp-7148
getCellVolume() ID: mp-3614
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-5308
getCellVolume() ID: mp-990430
getCellVolume() ID: mp-3614
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellV

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.



In [346]:
df["Volume_COD"]=df["COD_ID"].apply(getCellVolume)
df["Z_COD"]=df["COD_ID"].apply(getCellZValue)
df['Valence Electrons Density_COD'] = (df['Valence electrons'] * df['Z_COD']) / df['Volume_COD']
df[['a_COD', 'b_COD', 'c_COD']] = df['COD_ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526720
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526777
getCellVolume() ID: -1
getCellVolume() ID: 8101224


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: 7039788
getCellVolume() ID: 1526720
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526777
getCellVolume() ID: -1
getCellVolume() ID: 1522043
getCellVolume() ID: 7221084
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1522043
getCellVolume() ID: 7221084
getCellVolume() ID: -1
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: -1
getCellVolume() ID: 7221013
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 2102087
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 2102087
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1540895
getCellVolume() ID: -1
getCellVolume() ID: 2102087
getCel

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_

getCellVolume() ID: sd_1045310
getCellVolume() ID: sd_0548135
getCellVolume() ID: sd_0548134
getCellVolume() ID: sd_1045310
getCellVolume() ID: sd_0548135
getCellVolume() ID: sd_0548134
getCellVolume() ID: 1001177
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: -1
getCellVolume() ID: 1544433
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 2007573
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist

In [347]:
df["Volume_Springer"]=df["Springer_ID"].apply(getCellVolume)
df["Z_Springer"]=df["Springer_ID"].apply(getCellZValue)
df['Valence Electrons Density_Springer'] = (df['Valence electrons'] * df['Z_Springer']) / df['Volume_Springer']
df[['a_Springer', 'b_Springer', 'c_Springer']] = df['Springer_ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: -1
getCellVolume() ID: sd_1050391
getCellVolume() ID: sd_1050391
getCellVolume() ID: sd_1050391
getCellVolume() ID: -1
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0306445
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0306445


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

getCellVolume() ID: -1
getCellVolume() ID: sd_0376709
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1200159
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1200159


/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1041: UserWarning:

OH parsed as 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_spac

getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1810747
getCellVolume() ID: sd_1810747
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1810747
getCellVolume() ID: sd_1810747
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1237170
getCellVolume() ID: sd_1237170
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1820105
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1241784


...

getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1925412
ERROR: Invalid structure for  sd_1925412
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellV

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412


In [348]:
df["Volume_manual"]=df["a, A"]*df["b, A"]*df["c, A"]
df['Valence Electrons Density_manual'] = (df['Valence electrons'] * df['Z']) / df['Volume_manual']

##Oxygen concentration

In [349]:
def count_oxigen(formula):
  #match = re.search(r'O(\d+)', hill_formula)
  #if match:
  #  number = int(match.group(1))
  #  print(number)
  #  return number
  #else:
  #  print("No match found")
  if formula is None:
    return 0
  print(formula)
  comp = Composition(formula)

  # Get number of oxygen atoms
  oxygen_count = comp.get_el_amt_dict().get("O", 0)
  print(oxygen_count)
  return oxygen_count

In [350]:
df['Oxygen_count']=df['Hill formula'].apply(count_oxigen)

La O7 Rb Ta2
7.0
C3 H7 O7 Rb Ta2
7.0
Nd O7 Rb Ta2
7.0
O7 Rb Sm Ta2
7.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ba La4 O15 Ti4
15.0
Ba3 La Nb3 O12
12.0
Nb4 O15 Sr5
15.0
Ba5 Nb4 O15
15.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ca La4 O15 Ti4
15.0
Nb6 O17 Rb4
17.0
Ca2 Cs O10 Ta3
10.0
Ca Cs Na O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
C6 H16 Ca2 N O10 Ta3
10.0
Ca2 Cs O10 Ta3
10.0
Ca Cs Na O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
H2 O7 Sr Ta2
7.0
Li2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.0
O7 Rb2 Sr Ta2
7.0
K O3 Ta
3.0
H2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.0
K O3 Ta
3.0
K6 La2 O21 Ta6
21.0
H6 La2 O21 Ta6
21.0
La O9 Ta3
9.0
H2 O7 Sr Ta2
7.0
K O3 Ta
3.0
K6 La2 O21 Ta6
21.0
H6 La2 O21 Ta6
21.0
La O9 Ta3
9.0
H2 O7 Sr Ta2
7.0
K O3 Ta
3.0
O7 Sr3 Ti2
7.0
O7 Sr3 Ti2
7.0
Bi2 K La O9 Ta2


In [351]:
df['Oxygen_concentration_manual']=df['Z']*df['Oxygen_count']/df['Volume_manual']
df['Oxygen_concentration_MP']=df['Z_MP']*df['Oxygen_count']/df['Volume_MP']
df['Oxygen_concentration_COD']=df['Z_COD']*df['Oxygen_count']/df['Volume_COD']
df['Oxygen_concentration_Springer']=df['Z_Springer']*df['Oxygen_count']/df['Volume_Springer']

##Packing fraction


In [352]:
ionic_radii={"H+":0.02}
difficult_compunds_oxidation_states={
    "Ca Cs Na O10 Ta3":{"Ca":2,"Cs":1,"Na":1,"O":-2,"Ta":5},
    "Fe K2 La2 O10 Ti2":{"Fe":3, "K": 1,"La":3,"O":-2,"Ti":4},
    'Ca2 H Nb O10 Ta':{"Ca":2,"H":1,"Nb": 5,"Ta":5,"O":-2},
    'K2 O10 Sr Ta3':{"K":1,"O":-2,"Sr":2,"Ta":5},
    'H Nb O10 Sr2 Ta':{"H":1,"Nb":5,"O":-2,"Sr":2,"Ta":5},
}

In [353]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
def get_packing_fraction_from_cif(mp_id):
    print(mp_id)
    cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
    cif_path = os.path.join(cif_folder, f"{mp_id}.cif")

    if not os.path.isfile(cif_path):
        return np.nan  # Return NaN if file doesn't exist
    try:
      structure = Structure.from_file(cif_path)
    except:
      return 0
    comp = structure.composition
    #comp_oxi = comp.oxidation_state_guesses()
    #print(comp_oxi)


    # Extract radius for each site
    site_radii = []

    try:
      print('Calculating oxidation states')
      structure.add_oxidation_state_by_guess()
    except:
      print('Error')
      return


    for site in structure:
        #el = Element(site.species_string)
        try:
          specie = site.specie
        except AttributeError:
          return 0
        print(site.specie.ionic_radii)
        print(site.specie.oxi_state)
        print('Specie ionic radii ',site.specie.ionic_radii)
        print('Site oxi satte ',site.specie.oxi_state)
        rad =site.specie.ionic_radii.get(site.specie.oxi_state)
        print('got site')
        if(rad is not None):
          site_radii.append(rad)

    # Compute atomic volume per site
    atom_volumes = [(4/3) * np.pi * r**3 for r in site_radii]
    total_atom_volume = sum(atom_volumes)

    # Compute packing fraction
    packing_fraction = total_atom_volume / structure.volume
    print('OK')
    return packing_fraction
    #except:
        #return np.nan  # Return NaN if anything fails (e.g., missing radii)

In [354]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
from pymatgen.core.periodic_table import Species

unresolved_compunds = []

def get_packing_fraction_from_formula_and_cell_volume(hill_formula, V, Z):
    if V==0 or np.isnan(V) or Z==0 or np.isnan(Z):
        return np.nan
    print("Enter!")
    print(hill_formula)
    print(V)
    print(Z)
    comp = Composition(hill_formula)
    oxi_guesses = comp.oxi_state_guesses()
    if(len(oxi_guesses)==0):
      oxi_guesses = difficult_compunds_oxidation_states.get(hill_formula)
      if(oxi_guesses==None):
        unresolved_compunds.append(hill_formula)
        return np.nan
    else:
      oxi_guesses = oxi_guesses[0]

    V_ions_formula=0
    for el, amt in comp.items():
        symbol = el.symbol
        el_oxidation_state = oxi_guesses[symbol]
        #element = Element(el)
        print("Element:", symbol)
        print("Element ox state:", el_oxidation_state)
        specie = Species(symbol,el_oxidation_state)
        r = specie.ionic_radius
        if(r==np.nan or r==None):
          ion_formula = symbol
          if(el_oxidation_state!=0 and el_oxidation_state!=1 and el_oxidation_state!=-1):
             ion_formula =  ion_formula+str(el_oxidation_state)
          if(el_oxidation_state>0):
            ion_formula =  ion_formula+str("+")
          if(el_oxidation_state<0):
            ion_formula =  ion_formula+str("-")
          print("Local ionic radii table request for ",ion_formula)
          r= ionic_radii.get(ion_formula)
          if(r==None):
            r=0
        print("r: ", r)
        V_ion = (4/3) * np.pi * r**3
        V_ions_formula += (V_ion*amt)
        print(el," ",amt)

    packing_fraction= Z*V_ions_formula/V
    return packing_fraction
    print("Output!")

In [355]:
#fr = get_packing_fraction_from_formula_and_cell_volume("H2 La2 Ti3 O10",390.7,2)
#print("Fraction: ", fr)

#fr = get_packing_fraction_from_formula_and_cell_volume("O2 Ti",1183,30)
#print("Fraction: ", fr)

fr = get_packing_fraction_from_formula_and_cell_volume("Ca Cs Na O10 Ta3",228,1)
print("Fraction: ", fr)

Enter!
Ca Cs Na O10 Ta3
228
1
Element: Ca
Element ox state: 2
r:  1.14 ang
Ca   1.0
Element: Cs
Element ox state: 1
r:  1.81 ang
Cs   1.0
Element: Na
Element ox state: 1
r:  1.16 ang
Na   1.0
Element: O
Element ox state: -2
r:  1.26 ang
O   10.0
Element: Ta
Element ox state: 5
r:  0.78 ang
Ta   3.0
Fraction:  0.5584979567677797


In [356]:
#df["MP_packing_fraction"] = df["Materials Project ID"].apply(get_packing_fraction_from_cif)
#df["COD_packing_fraction"] = df["COD_ID"].apply(get_packing_fraction_from_cif)
#df["Springer_packing_fraction"] = df["Springer_ID"].apply(get_packing_fraction_from_cif)

df["MP_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_MP"], row["Z_MP"]), axis=1)
df["COD_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_COD"], row["Z_COD"]), axis=1)
df["Springer_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_Springer"], row["Z_Springer"]), axis=1)
df["Manual_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_manual"], row["Z"]), axis=1)

Enter!
La O7 Rb Ta2
173.01142785646587
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Rb
Element ox state: 1.0
r:  1.66 ang
Rb   1.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   2.0
Enter!
La4 O12 Ti3
238.90888175434102
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   12.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
Ba La4 O15 Ti4
623.8123076220189
2.0
Element: Ba
Element ox state: 2.0
r:  1.49 ang
Ba   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   15.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   4.0
Enter!
Ba3 La Nb3 O12
275.1761178624919
1.0
Element: Ba
Element ox state: 2.0
r:  1.49 ang
Ba   3.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   3.0
Element: O
Element ox state: -2.0
r:  

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for H+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Ta4.33+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Zr2+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C1.33-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C4-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for H0.31+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Ti3.67+!



r:  0.745 ang
Ti   2.0
Enter!
K2 La2 O10 Ti3
229.90248606119223
1.0
Element: K
Element ox state: 1.0
r:  1.52 ang
K   2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
H La2 O10 Pb S Ti3
229.90248606119223
1.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Element: Pb
Element ox state: 4.0
r:  0.915 ang
Pb   1.0
Element: S
Element ox state: -2.0
r:  1.7 ang
S   1.0
Element: Ti
Element ox state: 3.6666666666666665
Local ionic radii table request for  Ti3.6666666666666665+
r:  0
Ti   3.0
Enter!
La2 O7 Ti2
288.3342929018237
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
La2 O7

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Zr2+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C1.33-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for H+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C1.33-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for H+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Ta4.33+!



r:  1.52 ang
K   6.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   21.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   6.0
Enter!
H6 La2 O21 Ta6
347.64362841408706
1.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   6.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   21.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   6.0
Enter!
Bi2 K La O9 Ta2
760.635539456
2.0
Element: Bi
Element ox state: 3.0
r:  1.17 ang
Bi   2.0
Element: K
Element ox state: 1.0
r:  1.52 ang
K   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   9.0
Element: Ta
Element ox state: 4.0
r:  0.82 ang
Ta   2.0
Enter!
Bi2 K La Nb2 O9
760.635539456
2.0
Element: Bi
Element ox state: 3.0
r:  1.17 ang
Bi   2.0
Element: K
Element ox state: 1.0
r:  1.52 ang
K   1.0
Element: La
Element ox state: 3.0
r:

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C2-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for Nb2+!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for C4-!

/tmp/ipython-input-4269089585.py:32: UserWarning:

No ionic radius for H0.31+!



Element: C
Element ox state: -4.0
Local ionic radii table request for  C-4.0-
r:  0
C   3.0
Element: H
Element ox state: 0.3055555555555556
Local ionic radii table request for  H0.3055555555555556+
r:  0
H   72.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 2.0
r:  1.0 ang
Ti   2.0
Enter!
C3 H7 La O7 Ti2
556.266454565
4.0
Element: C
Element ox state: -1.3333333333333333
Local ionic radii table request for  C-1.3333333333333333-
r:  0
C   3.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   7.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
Bi2 Nb2 O9 Sr
757.1695024009999
4.0
Element: Bi
Element ox state: 3.0
r:  1.17 ang
Bi   2.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   9.0
Element: Sr
Element ox state: 2.0
r:  1.32 an

/tmp/ipython-input-4269089585.py:32: UserWarning:

No default ionic radius for Mn4+. Using ls data.



In [357]:
print(unresolved_compunds)

[]


##Averaging


In [358]:
def averaging_valence_electron_density(MP,COD,Springer):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  print('Count: ',count)
  if count==0:
    return 0
  return sum/count


def averaging_4(MP,COD,Springer,manual):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  print(manual)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  if(pd.notna(manual)):
    sum+=manual
    count+=1
  print('Count: ',count)
  if count==0:
    print("No value")
    return np.nan
  return sum/count

In [359]:
df['Valence Electrons Density avg'] = df.apply(lambda x: averaging_4(x['Valence Electrons Density_MP'], x['Valence Electrons Density_COD'],x['Valence Electrons Density_Springer'],x['Valence Electrons Density_manual']), axis=1)
df['Oxygen_concentration avg'] = df.apply(lambda x: averaging_4(x['Oxygen_concentration_MP'], x['Oxygen_concentration_COD'],x['Oxygen_concentration_Springer'],x['Oxygen_concentration_manual']), axis=1)
df['Packing fraction avg'] = df.apply(lambda x: averaging_4(x['MP_packing_fraction'], x['COD_packing_fraction'],x['Springer_packing_fraction'],x['Manual_packing_fraction']), axis=1)

0.08091951019336545
nan
nan
0.083414433015833
Count:  2
nan
nan
0.08508481120279507
0.08345739139442625
Count:  2
nan
nan
0.08508481120279507
0.08526808257143804
Count:  2
nan
nan
0.08508481120279507
0.08536881551918019
Count:  2
0.10045670895014315
0.10245478442963495
nan
nan
Count:  2
nan
nan
0.10275485551073929
nan
Count:  1
nan
nan
0.10162814320920892
nan
Count:  1
0.09618277688159252
0.09952839965001856
nan
nan
Count:  2
0.08721687109487106
nan
nan
nan
Count:  1
0.09000833274200813
0.09396027924991113
nan
nan
Count:  2
0.08487580957467307
0.08752221672887435
nan
nan
Count:  2
0.10045670895014315
0.10245478442963495
nan
nan
Count:  2
nan
nan
0.10275485551073929
nan
Count:  1
nan
nan
0.10162814320920892
nan
Count:  1
0.09618277688159252
0.09952839965001856
nan
nan
Count:  2
nan
nan
0.07792328933455918
nan
Count:  1
0.08260097161100627
0.08773052253977544
nan
0.08843641282630438
Count:  3
0.08741463890898929
0.09185591428801693
nan
0.07763894381844164
Count:  3
0.08741463890898929
na

In [360]:
df['Log_rate'] = np.log(df['Rate, umol/(g*h)'])
df['Log_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=["Log_rate"])
df['Log_rate'].value_counts()

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

/tmp/ipython-input-830342094.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,count
Log_rate,
1.609438,7
2.302585,6
1.791759,6
6.253829,5
3.374169,4
...,...
1.629641,1
5.091463,1
3.378840,1


EDA

In [361]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305 entries, 69 to 1070
Data columns (total 75 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Perovskite                          305 non-null    object 
 1   Interlayer space composition        5 non-null      object 
 2   Bandgap, eV                         305 non-null    float64
 3   Materials Project ID                305 non-null    object 
 4   COD_ID                              305 non-null    object 
 5   Springer_ID                         305 non-null    object 
 6   Z                                   225 non-null    float64
 7   a, A                                226 non-null    float64
 8   b, A                                225 non-null    float64
 9   c, A                                225 non-null    float64
 10  Symmetry group                      187 non-null    object 
 11  d,A                                 240 non-null

In [362]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)', 'Hill formula', 'avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'Average Mulliken electronegativity',
       'Valence electrons', 'Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Elec

In [363]:
columns_EDA = ['Perovskite','Bandgap, eV',
       'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'Average Mulliken electronegativity', 'Valence electrons',
       'Valence Electrons Density avg', 'Oxygen_concentration avg',
       'Packing fraction avg','Log_rate']

In [364]:
categorical_cols = [col for col in columns_EDA
                    if df[col].dtype in ['object', 'category','bool']]

numerical_cols = [col for col in columns_EDA
                  if df[col].dtype in ['int64', 'float64']]
print(categorical_cols)
print(numerical_cols)

['Perovskite', 'Symmetry group', 'Prep Method', 'Nitrogen', 'Promotion method', 'Promoter', 'Light type', 'Cell material']
['Bandgap, eV', 'd,A', 'Number of octahedrons on a layer', 'CalcT(K)', 'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Power, W', 'Wave length min, nm', 'Temperature, K', 'Rate, umol/(g*h)', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'Average Mulliken electronegativity', 'Valence electrons', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Log_rate']


In [365]:
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

def plot_smooth_distribution(df, column):
    data = df[column].dropna().values

    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 500)
    y_vals = kde(x_vals)

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(width=3),
        name=f'{column} KDE'
    ))

    fig.update_layout(
        title=f"Distribution of '{column}'",
        xaxis_title=column,
        yaxis_title="Density",
        #template="plotly_white"
    )

    fig.show()

In [366]:
plot_smooth_distribution(df, 'Bandgap, eV')

In [367]:
for col in numerical_cols:
    fig = px.histogram(df, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()

In [368]:
def draw_pie_chart(df, column):
  fig = px.pie(df, names=column, title=f'{column} Distribution',hole=0.3)
  fig.update_traces(textinfo='percent+label')
  fig.show()

In [369]:
for col in categorical_cols:
     draw_pie_chart(df,col)

In [370]:
from scipy.stats import pearsonr

def draw_scatter_plot(df,x_column, y_column):
  from sklearn.metrics import r2_score

  x = df[x_column]
  y = df[y_column]

  r, p_value = pearsonr(x, y)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      mode='markers',
      name='Data',
      marker=dict(size=8, color='blue', opacity=0.7)
  ))

  fig.add_annotation(
    x=np.mean(x),
    y=np.max(y),
    text=f"Pearson r = {r:.3f}",
    showarrow=False,
    font=dict(size=14, color="red")
  )

  fig.update_layout(
      xaxis_title=f'{x_column}',
      yaxis_title=f'{y_column}',
      template="plotly_white",
      width=600,
      height=600,
  )

  fig.show()

In [371]:
for col in numerical_cols:
  draw_scatter_plot(df,col,'Rate, umol/(g*h)')

In [372]:
for col in numerical_cols:
  draw_scatter_plot(df,col,target_col)

In [373]:
df_selected = df[numerical_cols]

def draw_corr_heatmap(_df):
  # Compute correlation matrix
  corr_matrix = _df.corr()

  # Plot heatmap
  fig = px.imshow(
      corr_matrix,
      text_auto=True,        # show correlation values
      aspect="auto",
      color_continuous_scale='RdBu_r',  # diverging color map
      title="Correlation Heatmap"
  )
  fig.show()

draw_corr_heatmap(df_selected)

In [374]:
df.to_excel('checkpoint_descriptors.xlsx')

#Preparing data for ML

##Categorization


In [375]:
#raise SystemExit("Stopping execution")

In [376]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns.remove('Perovskite')
categorical_columns.remove('Interlayer space composition')
categorical_columns.remove('Materials Project ID')
categorical_columns.remove('COD_ID')
categorical_columns.remove('Springer_ID')
categorical_columns.remove('Symmetry group')
categorical_columns.remove('Sac Agent 1')
categorical_columns.remove('Sac agent 2')
categorical_columns.remove('Hill formula')
categorical_columns

['Prep Method', 'Promotion method', 'Promoter', 'Light type', 'Cell material']

In [377]:
df2 = df['Light type'].value_counts()
print(df2.index)

Index(['Hg lamp', 'Xe lamp'], dtype='object', name='Light type')


In [378]:
df[categorical_columns].isna().sum()

,0
Prep Method,0
Promotion method,0
Promoter,0
Light type,0
Cell material,0


In [379]:
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df.columns


Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Power, W', 'Wave length min, nm', 'Temperature, K',
       'Rate, umol/(g*h)', 'Hill formula', 'avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'Average Mulliken electronegativity',
       'Valence electrons', 'Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer'

In [380]:
columns_for_ML = df.columns
columns_to_exlude = ['Perovskite', 'Interlayer space composition','Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'Number of octahedrons on a layer','Sac Agent 1',
       'Sac agent 2','Rate, umol/(g*h)','Hill formula','Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer', 'Valence Electrons Density_Springer',
       'a_Springer', 'b_Springer', 'c_Springer', 'Volume_manual',
       'Valence Electrons Density_manual', 'Oxygen_count',
       'Oxygen_concentration_manual', 'Oxygen_concentration_MP',
       'Oxygen_concentration_COD', 'Oxygen_concentration_Springer',
       'MP_packing_fraction', 'COD_packing_fraction',
       'Springer_packing_fraction', 'Manual_packing_fraction']

columns_for_ML = [x for x in columns_for_ML if x not in columns_to_exlude]
print(columns_for_ML)


['Bandgap, eV', 'd,A', 'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Power, W', 'Wave length min, nm', 'Temperature, K', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'Average Mulliken electronegativity', 'Valence electrons', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Log_rate', 'Prep Method_MSS', 'Prep Method_PC', 'Prep Method_PTP', 'Prep Method_SG', 'Prep Method_SSR', 'Promotion method_PD', 'Promotion method_impregnation', 'Promoter_No promoter', 'Promoter_Pt', 'Light type_Xe lamp', 'Cell material_quartz']


In [381]:
df = df[columns_for_ML]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305 entries, 69 to 1070
Data columns (total 37 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Bandgap, eV                         305 non-null    float64
 1   d,A                                 240 non-null    float64
 2   CalcT(K)                            305 non-null    float64
 3   Calc time (h)                       305 non-null    float64
 4   Nitrogen                            305 non-null    bool   
 5   Promoter, w%                        305 non-null    float64
 6   Surface area, m2/g                  305 non-null    float64
 7   CatW, g/L                           305 non-null    float64
 8   Alcohol, %                          305 non-null    float64
 9   Power, W                            305 non-null    float64
 10  Wave length min, nm                 305 non-null    float64
 11  Temperature, K                      305 non-null

In [382]:
df["Valence Electrons Density avg"] = df["Valence Electrons Density avg"].fillna(df["Valence Electrons Density avg"].mean())
df["Oxygen_concentration avg"] = df["Oxygen_concentration avg"].fillna(df["Oxygen_concentration avg"].mean())
df["Packing fraction avg"] = df["Packing fraction avg"].fillna(df["Packing fraction avg"].mean())

In [383]:
df["d,A"] = df["d,A"].fillna(df["d,A"].mean())
print(df["d,A"].mean())

16.627029999999998


In [384]:
df.isna().sum().sum()

np.int64(0)

In [385]:
draw_corr_heatmap(df)

In [386]:
features_to_remove=['Oxygen_concentration avg','Valence electrons']
df = df.drop(columns=features_to_remove)

##Normalization

In [387]:
from sklearn.preprocessing import MinMaxScaler

features = list(df.columns)
features.remove('Log_rate')

scaler = MinMaxScaler()

df_normalized = df.copy()
df_normalized[features] = scaler.fit_transform(df_normalized[features])
df_normalized.head()

,"Bandgap, eV","d,A",CalcT(K),Calc time (h),Nitrogen,"Promoter, w%","Surface area, m2/g","CatW, g/L","Alcohol, %","Power, W",...,Prep Method_PC,Prep Method_PTP,Prep Method_SG,Prep Method_SSR,Promotion method_PD,Promotion method_impregnation,Promoter_No promoter,Promoter_Pt,Light type_Xe lamp,Cell material_quartz
69,0.743590,0.321713,0.818182,0.240506,0.0,0.0,0.638564,0.218750,0.0,0.846154,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
70,0.538462,0.321713,0.818182,0.240506,0.0,0.0,0.638564,0.218750,0.0,0.846154,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
71,0.692308,0.321134,0.818182,0.240506,0.0,0.0,0.638564,0.218750,0.0,0.846154,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
72,0.692308,0.321423,0.818182,0.240506,0.0,0.0,0.638564,0.218750,0.0,0.846154,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
83,0.769231,0.481037,0.909091,0.240506,0.0,0.0,0.271186,0.302083,0.0,0.538462,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [388]:
draw_corr_heatmap(df_normalized)

In [389]:
X = df_normalized.drop(columns=[target_col])
y = df_normalized[target_col]

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.9
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print("Columns to drop:", to_drop)
df_reduced = df.drop(columns=to_drop)

Columns to drop: ['Wave length min, nm', 'Temperature, K', 'avg p valence electrons', 'frac f valence electrons', 'Promoter_Pt', 'Cell material_quartz']


In [390]:
df_normalized.to_excel("checkpoint_ML_input.xlsx")

#ML


In [392]:
#raise SystemExit("Stopping execution")


print('Std:',y.std())

print(X.shape[0])

Std: 2.932536490594177
305


In [393]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

In [394]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
)


In [395]:
def draw_actual_predicted(_y_test, _y_pred):
  fig = px.scatter(
    x=_y_test,
    y=_y_pred,
    labels={'x':'Actual', 'y':'Predicted'},
    title='Actual vs Predicted',
    width = 700,
    height = 700,
  )
  fig.add_shape(
      type="line",
      x0=_y_test.min(), y0=_y_pred.min(),
      x1=_y_test.max(), y1=_y_pred.max(),
      line=dict(color="red", dash="dash")
  )
  rmse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  fig.add_annotation(
    x=0.05, y=0.95, xref="paper", yref="paper",
    text=f"R² = {r2:.4f}<br>RMSE = {rmse:.4f}",
    showarrow=False,
    font=dict(size=14)
  )
  fig.show()

##Linear regression

In [396]:
linear_model = LinearRegression()

if(cross_validation):
  cv = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(linear_model, X, y, cv=cv, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(linear_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  linear_model.fit(X_train, y_train)
  y_pred = linear_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [ -3.06481554 -14.13923465  -2.36644146  -3.23887381  -2.52133724]
Mean RMSE: 2.0873524255395735


In [397]:
def estimate_performance(_y_test, _y_pred):
  raisemse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  print("RMSE:", mse)
  print("R² Score:", r2)

##Decision tree regression

In [398]:
from sklearn.tree import DecisionTreeRegressor

In [399]:
tree_model = DecisionTreeRegressor(random_state=42)

if(cross_validation):
  scores = cross_val_score(tree_model, X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(tree_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  tree_model.fit(X_train, y_train)
  y_pred = tree_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-10.23191631  -7.77194814  -8.59402032  -2.07709831  -9.4568632 ]
Mean RMSE: 2.6869059067774934


In [400]:
import matplotlib.pyplot as plt

if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': tree_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

## Random forest

In [401]:
from sklearn.ensemble import RandomForestRegressor

In [402]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=3,
    min_samples_split = 5,
    min_samples_leaf = 3,
    max_features=2,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(rf_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(rf_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  rf_model.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.5189633  -7.32364818 -7.4103582  -3.69439439 -6.75491027]
Mean RMSE: 2.45975408764294


In [403]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': rf_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [404]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 5],
    'max_features': [1, 2, 3]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                           param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X,y)
print('Best params',grid_search.best_params_)
print('Best score',grid_search.best_score_)

Best params {'max_depth': 5, 'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Best score -5.180021083902263


In [405]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-6.13562022 -6.09810286 -6.88897728 -2.5748653  -4.20253976]
Mean RMSE: 2.2451573270417513


##Gradient boosting

In [406]:
from sklearn.ensemble import GradientBoostingRegressor

In [407]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,      # number of boosting stages
    learning_rate=0.1,     # shrinkage factor
    max_depth=3,           # depth of each tree
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(gb_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(gb_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  gb_model.fit(X_train, y_train)
  y_pred = gb_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-7.01810096 -4.93262639 -5.89130082 -1.81425992 -4.8957449 ]
Mean RMSE: 2.171379887338808


In [408]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': gb_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [409]:
param_grid = {
    'n_estimators': [100, 150],        # number of boosting stages
    'learning_rate': [0.01, 0.1],   # shrinkage factor
    'max_depth': [2, 3, 4],                # depth of each tree
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 1.0]                # fraction of samples for each tree
}

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',          # or 'neg_mean_squared_error'
    n_jobs=-1,
    verbose=2
)

# Fit on training data
grid_search.fit(X, y)

# Best hyperparameters
print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}
Best CV RMSE: -4.570203959186105


In [410]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-6.20311133 -4.65627393 -5.82765886 -1.93307523 -4.23090044]
Mean RMSE: 2.1019530391577232


##KNN

In [411]:
from sklearn.neighbors import KNeighborsRegressor

In [412]:
knn_model = KNeighborsRegressor(n_neighbors=5)

if(cross_validation):
  scores = cross_val_score(knn_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(knn_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  knn_model.fit(X_train, y_train)
  y_pred = knn_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-8.67755442 -6.99862009 -9.06254639 -4.48879535 -3.31489275]
Mean RMSE: 2.50820564030332


In [413]:
param_grid = {
    'n_neighbors': list(range(1, 21)),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(
    KNeighborsRegressor(),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Best CV RMSE: -5.9360502840841916


In [414]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-8.63495182 -5.21509697 -9.16917267 -4.53923763 -2.12179233]
Mean RMSE: 2.3674875433757543


##Catboost

In [415]:
!pip install catboost
from catboost import CatBoostRegressor

In [416]:
cat_boost_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=100
)

if(cross_validation):
  scores = cross_val_score(cat_boost_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(cat_boost_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  cat_boost_model.fit(X_train, y_train)
  y_pred = cat_boost_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

0:	learn: 2.8095920	total: 1.91ms	remaining: 1.91s
100:	learn: 0.6093997	total: 140ms	remaining: 1.25s
200:	learn: 0.4006400	total: 506ms	remaining: 2.01s
300:	learn: 0.2780532	total: 929ms	remaining: 2.16s
400:	learn: 0.2077620	total: 1.09s	remaining: 1.64s
500:	learn: 0.1534762	total: 1.24s	remaining: 1.24s
600:	learn: 0.1136180	total: 1.5s	remaining: 994ms
700:	learn: 0.0872743	total: 1.73s	remaining: 739ms
800:	learn: 0.0687518	total: 2.09s	remaining: 519ms
900:	learn: 0.0573175	total: 2.63s	remaining: 288ms
999:	learn: 0.0475515	total: 3.1s	remaining: 0us
0:	learn: 2.6567771	total: 7.79ms	remaining: 7.78s
100:	learn: 0.7219224	total: 548ms	remaining: 4.88s
200:	learn: 0.5412921	total: 1.06s	remaining: 4.21s
300:	learn: 0.4786008	total: 1.59s	remaining: 3.69s
400:	learn: 0.4426224	total: 2.32s	remaining: 3.46s
500:	learn: 0.4253205	total: 2.9s	remaining: 2.89s
600:	learn: 0.4174030	total: 3.39s	remaining: 2.25s
700:	learn: 0.4127047	total: 4.16s	remaining: 1.77s
800:	learn: 0.40969

In [417]:
param_grid = {
    'depth': [4,6,8],
    'learning_rate':[0.01,0.05,0.1 ]
}

grid_search = GridSearchCV(cat_boost_model, param_grid, cv=5,scoring='neg_mean_squared_error',)
grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

0:	learn: 2.9597956	total: 738us	remaining: 738ms
100:	learn: 1.8919804	total: 59.3ms	remaining: 528ms
200:	learn: 1.4236933	total: 116ms	remaining: 462ms
300:	learn: 1.2067437	total: 234ms	remaining: 542ms
400:	learn: 1.0846918	total: 344ms	remaining: 514ms
500:	learn: 1.0001308	total: 464ms	remaining: 463ms
600:	learn: 0.9298740	total: 678ms	remaining: 450ms
700:	learn: 0.8738487	total: 875ms	remaining: 373ms
800:	learn: 0.8311687	total: 942ms	remaining: 234ms
900:	learn: 0.7989727	total: 998ms	remaining: 110ms
999:	learn: 0.7659152	total: 1.05s	remaining: 0us
0:	learn: 2.7896790	total: 1.05ms	remaining: 1.05s
100:	learn: 1.8113570	total: 85.1ms	remaining: 758ms
200:	learn: 1.4175111	total: 153ms	remaining: 607ms
300:	learn: 1.2470403	total: 220ms	remaining: 512ms
400:	learn: 1.1578952	total: 301ms	remaining: 450ms
500:	learn: 1.0917663	total: 477ms	remaining: 476ms
600:	learn: 1.0322948	total: 578ms	remaining: 384ms
700:	learn: 0.9805188	total: 675ms	remaining: 288ms
800:	learn: 0.9

In [418]:
best_model = grid_search.best_estimator_

if(cross_validation):
  print("Cross validation")
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Cross validation
0:	learn: 2.8919566	total: 4.76ms	remaining: 4.75s
100:	learn: 0.8283924	total: 314ms	remaining: 2.79s
200:	learn: 0.6028874	total: 724ms	remaining: 2.88s
300:	learn: 0.4844229	total: 1.05s	remaining: 2.45s
400:	learn: 0.3968897	total: 1.5s	remaining: 2.23s
500:	learn: 0.3252017	total: 1.91s	remaining: 1.9s
600:	learn: 0.2726741	total: 2.35s	remaining: 1.56s
700:	learn: 0.2333214	total: 2.88s	remaining: 1.23s
800:	learn: 0.2004742	total: 3.42s	remaining: 851ms
900:	learn: 0.1718511	total: 3.85s	remaining: 423ms
999:	learn: 0.1478344	total: 4.46s	remaining: 0us
0:	learn: 2.7294863	total: 8.96ms	remaining: 8.95s
100:	learn: 0.9507652	total: 826ms	remaining: 7.35s
200:	learn: 0.7233536	total: 1.41s	remaining: 5.6s
300:	learn: 0.6076038	total: 2.21s	remaining: 5.13s
400:	learn: 0.5401046	total: 2.92s	remaining: 4.37s
500:	learn: 0.5002878	total: 3.52s	remaining: 3.5s
600:	learn: 0.4716137	total: 4.13s	remaining: 2.74s
700:	learn: 0.4547784	total: 4.61s	remaining: 1.96s
800

Scores:  [-6.06432231 -3.81768801 -4.90925243 -2.49918889 -3.13732905]
Mean RMSE: 1.9968582518935452


In [419]:
y_pred = cross_val_predict(best_model , X, y, cv=5)

0:	learn: 2.8919566	total: 1.65ms	remaining: 1.65s
100:	learn: 0.8283924	total: 135ms	remaining: 1.2s
200:	learn: 0.6028874	total: 268ms	remaining: 1.06s
300:	learn: 0.4844229	total: 403ms	remaining: 936ms
400:	learn: 0.3968897	total: 547ms	remaining: 818ms
500:	learn: 0.3252017	total: 683ms	remaining: 680ms
600:	learn: 0.2726741	total: 821ms	remaining: 545ms
700:	learn: 0.2333214	total: 978ms	remaining: 417ms
800:	learn: 0.2004742	total: 1.11s	remaining: 277ms
900:	learn: 0.1718511	total: 1.25s	remaining: 137ms
999:	learn: 0.1478344	total: 1.39s	remaining: 0us
0:	learn: 2.7294863	total: 2.07ms	remaining: 2.06s
100:	learn: 0.9507652	total: 178ms	remaining: 1.58s
200:	learn: 0.7233536	total: 354ms	remaining: 1.41s
300:	learn: 0.6076038	total: 560ms	remaining: 1.3s
400:	learn: 0.5401046	total: 751ms	remaining: 1.12s
500:	learn: 0.5002878	total: 992ms	remaining: 988ms
600:	learn: 0.4716137	total: 1.17s	remaining: 778ms
700:	learn: 0.4547784	total: 1.35s	remaining: 578ms
800:	learn: 0.4410

In [420]:
draw_actual_predicted(y, y_pred)